<a href="https://colab.research.google.com/github/LeeYuuuan/DataMiningProject/blob/main/DataMining_ChiMerge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from sklearn import datasets
import pandas as pd
import numpy as np


In [17]:
iris = datasets.load_iris()
data=iris['data']
target=iris['target']
targetname=iris['target_names']
df=pd.DataFrame(data,columns=["Sepal_Length","Sepal_Width","Petal_Length","Petal_Width"])
df["is_Iris_Setosa(labels_0)"]=target==0
df["is_Iris_Versicolour(labels_1)"]=target==1
df["is_Iris_Virginica(labels_2)"]=(target==2)
df=df.replace(True,1)
df=df.replace(False,0)

In [24]:
def getOneAttributeData(df,attribute):
  """
  get a DataFrame with only one attribute
  df: a DataFrame with multiple attribute and one-hot vector.
  attribute: which attribute to get.
  """
  x=df.sort_values(attribute)
  return x[[attribute,"is_Iris_Setosa(labels_0)","is_Iris_Versicolour(labels_1)","is_Iris_Virginica(labels_2)"]].reset_index()

def GetInitializedInterval(value,left,right):
  """
  get a Interval for a value in initialization
  value: Current value.
  left: the value before Current value.
  right: the value after Current value.
  """
  left_Int=(left+value)/2
  right_Int=(right+value)/2
  return left_Int, right_Int

def GetInitializedIData(df):
  """
  Initialize data using original data.
  create a new dataframe
  columns: 
  Interval_left,Interval_right:     to indicate Interval
  dataIndex:                        to store the Index of the data in this interval.
  frequency_Setosa:                 the occurrence of the Setosa
  frequency_Versicolour:            the occurrence of the Versicolour
  frequency_Virginica:              the occurrence of the Virginica
  chi_square_with_next:             the chi square between this interval and next one.
  """
  InitializedDf=pd.DataFrame(columns=["Interval_left","Interval_right","dataIndex","frequency_Setosa","frequency_Versicolour","frequency_Virginica","chi_square_with_next"])
  Index=[]
  frequency_Setosa=0
  frequency_Versicolour=0
  frequency_Virginica=0
  for i in range(df.shape[0]):
    frequency_Setosa+=df[df.columns[2]][i]
    frequency_Versicolour+=df[df.columns[3]][i]
    frequency_Virginica+=df[df.columns[4]][i]
    Index.append(i)
    if i<df.shape[0]-1:
      if df[df.columns[1]][i+1]==df[df.columns[1]][i]:
        continue
      else:

        if i==0:
          left=-df[df.columns[1]][0]
        else:
          left=df[df.columns[1]][i-1]
        if i==df.shape[0]-1:
          right==-df[df.columns[1]][i]+1 
        else:
          right=df[df.columns[1]][i+1]
          value=df[df.columns[1]][i]
          Interval_left,Interval_right=GetInitializedInterval(value,left,right)
          InitializedDf=InitializedDf.append([{"Interval_left":Interval_left,"Interval_right":Interval_right,"dataIndex":Index,"frequency_Setosa":frequency_Setosa,"frequency_Versicolour":frequency_Versicolour,"frequency_Virginica":frequency_Virginica}])
          Index=[]
          frequency_Setosa=0
          frequency_Versicolour=0
          frequency_Virginica=0
    if i == df.shape[0]-1:
      right==-df[df.columns[1]][i]+1
      Interval_left,Interval_right=GetInitializedInterval(value,left,right)
      InitializedDf=InitializedDf.append([{"Interval_left":Interval_left,"Interval_right":right,"dataIndex":Index,"frequency_Setosa":frequency_Setosa,"frequency_Versicolour":frequency_Versicolour,"frequency_Virginica":frequency_Virginica}])
  return InitializedDf

def cal_chi_square(Interval,Interval_next):
  """
  Calculate the chi square between two intervals.
  Interval represents the current interval.
  Interval_next represents the next interval.
  The function returns a chi square value between two intervals.

  """
  contingency=GetContingency(Interval,Interval_next)
  ChiSquareValue=0
  for i in range(2):
    for j in range(3):
      temp=contingency[i][3]*contingency[2][j]/contingency[2][3]
      if temp==0:
        ChiSquareValue+=0
      else:
        ChiSquareValue+=(contingency[i][j]-temp)**2/temp
  return ChiSquareValue

def GetContingency(Interval_1,Interval_2):
  """
  Get the Contingency between Interval_1 and Interval_2.
  Both Interval_1 and Interval_2 are pandas dataframe.
  The function returns a contingency table of these two interval.
  
  """
  contingency=np.zeros([3,4])
  for i in range(3):
    contingency[0][i]=Interval_1[Interval_1.columns[i+3]][0]
    contingency[1][i]=Interval_2[Interval_2.columns[i+3]][0]
  for i in range(3):
    contingency[2][i]=contingency[0][i]+contingency[1][i]
  for i in range(2):
    contingency[i][3]=contingency[i][0]+contingency[i][1]+contingency[i][2]
  contingency[2][3]=contingency[2][0]+contingency[2][1]+contingency[2][2]
  return contingency


def GetChiSquare(df):
  """
  Get the dataframe with Chi Square values not null.
  Input a dataframe without chi square value.
  The function returns a dataframe with calculated chi square values.
  """
  
  for i in range(df.shape[0]-1):
    df["chi_square_with_next"][i:i+1][0]=cal_chi_square(df[i:i+1],df[i+1:i+2])
  return df

def MergeInterval(df,index):
  """
  This function merge the Specified Intervals via index.
  Returns a merged dataframe.
  """
  newdf=pd.DataFrame()
  df["Interval_right"][index:index+1][0]=df["Interval_right"][index+1:index+2][0]
  df["dataIndex"][index:index+1][0].extend(df["dataIndex"][index+1:index+2][0])
  df["frequency_Setosa"][index:index+1][0]+=df["frequency_Setosa"][index+1:index+2][0]
  df["frequency_Versicolour"][index:index+1][0]+=df["frequency_Versicolour"][index+1:index+2][0]
  df["frequency_Virginica"][index:index+1][0]+= df["frequency_Virginica"][index+1:index+2][0]
  x=df[0:index+1].append(df[index+2:])
  return x

def GetMergedData(df):
  """
  This function merges all Intervals which should be merged.
  Returns a merged dataframe which cannot be mereged anymore.
  """
  # 2 degrees of freedom Chi_square=4.6(90%)
  len_df=df.shape[0]
  count=0
  chi_min=df.min()["chi_square_with_next"]
  chi=4.6
  """
  for i in range(len_df-2):
    if df["chi_square_with_next"][count:count+1][0]==chi_min and df["chi_square_with_next"][count:count+1][0]<chi:
      df=MergeInterval(df,count)
    else:
      count+=1
  """
  if True:
  #if count==(len_df-2):

    count=0
    for i in range(len_df-1):
      if count<df.shape[0]-1:
        if df["chi_square_with_next"][count:count+1][0]<chi:
          df=MergeInterval(df,count)
          count+=1
        else:
          count+=1
  return GetChiSquare(df)

def ChiMerge(df):
  if df.shape[0]<=6:
    return df
  else:
    return ChiMerge(GetMergedData(df))



In [26]:
def getResult(df):
  """
  This function use the dataset to get the results.
  Which print 4 tables representing 4 merged results.
  """
  for attri in df.columns[0:4]:
    getdf=getOneAttributeData(df,attri)
    Initdf=GetInitializedIData(getdf)
    Chidf=GetChiSquare(Initdf)
    result=ChiMerge(Chidf)
    result.rename(columns={"Interval_left":"Int_l","Interval_right":"Int_r",	"frequency_Setosa":"Setosa","frequency_Versicolour":"Versicolour","frequency_Virginica":"Virginica","chi_square_with_next":"chi"},inplace=True)
    print("The result of ",attri, " is")
    print(result[["Int_l","Int_r",	"Setosa","Versicolour","Virginica","chi"]])



In [20]:
getResult(df)

The result of  Sepal_Length  is
   Int_l  Int_r Setosa Versicolour Virginica        chi
0    0.0   5.05     28           3         1   0.971389
0    5.1   5.45     17           3         0  21.524304
0    5.5   5.85      5          18         5  13.551007
0    5.9   6.65      0          20        22   2.477803
0    6.7   7.50      0           6        16   2.082645
0    7.5   7.90      0           0         6        0.0
The result of  Sepal_Width  is
   Int_l  Int_r Setosa Versicolour Virginica        chi
0   0.00   2.45      1           9         1   8.245614
0   2.50   2.85      0          18        18  11.563227
0   2.90   3.25     16          21        23  16.674088
0   3.30   3.65     20           2         6   1.111111
0   3.70   4.05     10           0         2   0.576923
0   4.05   4.40      3           0         0        0.0
The result of  Petal_Length  is
   Int_l  Int_r Setosa Versicolour Virginica        chi
0    0.0   1.80     48           0         0  49.550769
0    1.9 

xx.shape[0]